# Pipeline


In [ ]:
# import data
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline

## assigned data
df = pd.read_csv("./data/housing.csv")

In [ ]:
###  split data to train
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns="median_house_value"),  # features
    df["median_house_value"],  # target
    stratify=pd.cut(
        df["median_income"],
        bins=[0.0, 1.5, 3.0, 4.5, 6.0, np.inf],
        labels=[1, 2, 3, 4, 5],
    ),
    test_size=0.2,
    random_state=42,
)

In [ ]:
# transformer1 = make_column_transformer(
#     (StandardScaler(), make_column_selector(dtype_include=np.number)),
#     (OneHotEncoder(), make_column_selector(dtype_include=object)),
# )
# pipeline = make_pipeline(
#     transformer1, KNNImputer(n_neighbors=k_value)
# )  # creamos un pipeline con las transformaciones previas y la imputación

# X_train_imputed_b = pipeline.fit_transform(
#     X_train
# )  # Aplicamos el pipeline a los datos de entrenamiento